In [ ]:

# Data Manipulation
import pandas as pd
import re
from datetime import datetime
# Data Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# Statistical Modeling
import statsmodels.api as sm
# Stopwords and lemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# Natural Language Processing (NLP)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
# Web Scraping
from bs4 import BeautifulSoup
# NLP with Pre-trained Models
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




Bert Model 

In [ ]:
# Path to the Excel file
file_path = r"C:\Users\Dom3n\Downloads\Headlines.xlsx"
# Display  of the first few rows of the dataframe
df = pd.read_excel(file_path)
df.head()

In [ ]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stop words and lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Join words back into a single string
    cleaned_text = " ".join(words)
    
    return cleaned_text

# Apply the cleaning function to the 'title' column
df['cleaned_title'] = df['title'].apply(clean_text)

In [ ]:

# Tokenize the headlines into sentences and then into words using NLTK
df['Sentences'] = df['cleaned_title'].apply(sent_tokenize)
df['Tokens'] = df['Sentences'].apply(lambda sents: [word_tokenize(sent) for sent in sents])

# Display the cleaned and tokenized data
df.head()


In [ ]:
# Load pre-trained model and tokenizer
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Function to get logits for each sentence
def get_logits(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits.detach().numpy()[0]
    return logits

# Calculate sentiment for each sentence and then average for each article
def calculate_sentiment(sentences):
    sentiments = []
    for sentence in sentences:
        logits = get_logits(sentence)
        sentiment = logits[4] - logits[0]  # Assuming logits[4] is for positive and logits[0] is for negative
        sentiments.append(sentiment)
    return sum(sentiments) / len(sentiments) if sentiments else 0

df['Sentiment Score'] = df['Sentences'].apply(calculate_sentiment)

In [ ]:
# Display the first 10 rows with sentiment scores
df.head(10)

In [ ]:
#Data split 
# Split data into training, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Display the number of samples in each set
print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

# Print percentage split
total_samples = len(df)
train_percentage = (len(train_df) / total_samples) * 100
val_percentage = (len(val_df) / total_samples) * 100
test_percentage = (len(test_df) / total_samples) * 100

print(f"Training set: {train_percentage:.2f}%")
print(f"Validation set: {val_percentage:.2f}%")
print(f"Test set: {test_percentage:.2f}%")


Data Analysis of Sentiment Analysis 

In [ ]:
# Extract the month and year from the pubDate column
df['month_year'] = df['date'].dt.strftime('%m')

# Sum the news titles per month
news_per_month = df.groupby('month_year')['title'].count()

# Create the histogram
plt.figure(figsize=(10, 6))
news_per_month.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Number Headlines')
plt.xticks(rotation=90)

plt.xticks(range(len(news_per_month)), news_per_month.index)
plt.gca().set_xticklabels(['January', 'February', 'March', 'April', 'May', 'June', 'July'])

plt.show()

In [ ]:
# Calculate the average sentiment per day
avg_sentiment_per_day = df.groupby('date')['Sentiment Score'].mean().reset_index()

In [ ]:
#Plot of the average sentiment per day 
plt.figure(figsize=(12, 6))
plt.plot(avg_sentiment_per_day['date'], avg_sentiment_per_day['Sentiment Score'], marker='o', linestyle='-')
plt.xlabel('Date')
plt.ylabel('Average Sentiment Score')
plt.xticks(rotation=45)
plt.grid(True)


In [ ]:

df['date'] = pd.to_datetime(df['date'])
# Sentiment per day 
average_sentiment_per_day = df.groupby('date')['Sentiment Score'].mean().reset_index()
average_sentiment_per_day.rename(columns={'Sentiment Score': 'Average Sentiment'}, inplace=True)
# Average sentiment per day 
print(average_sentiment_per_day.head())

In [ ]:
# Plot histogram of sentiment distribution
plt.figure(figsize=(10, 5))
sns.histplot(df['Sentiment Score'], bins=20, kde=True, color='blue')
plt.title('Distribution of Sentiment Scores in News Headlines')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plot sentiment categories distribution
plt.figure(figsize=(10, 5))
df['Sentiment Category'] = pd.cut(df['Sentiment Score'], bins=[-float('inf'), -1, 1, float('inf')], labels=['Negative', 'Neutral', 'Positive'])
sentiment_counts = df['Sentiment Category'].value_counts()
total_count = sentiment_counts.sum()

# Calculate the percentages
negative_percent = (sentiment_counts['Negative'] / total_count) * 100
neutral_percent = (sentiment_counts['Neutral'] / total_count) * 100
positive_percent = (sentiment_counts['Positive'] / total_count) * 100

# Plot the bar chart with percentages
sentiment_counts.plot(kind='bar', color=['slategrey', 'lightsteelblue', 'cornflowerblue'])
plt.xlabel('Sentiment Category')
plt.ylabel('Frequency')
plt.xticks(rotation=45)

# Add the percentage labels
for i, v in enumerate(sentiment_counts):
    if i == 0:
        plt.text(i, v, f"{v} ({negative_percent:.2f}%)", color='black', va='center', ha='center')
    elif i == 1:
        plt.text(i, v, f"{v} ({neutral_percent:.2f}%)", color='black', va='center', ha='center')
    else:
        plt.text(i, v, f"{v} ({positive_percent:.2f}%)", color='black', va='center', ha='center')

plt.show()

Descriptive statistics of dependent variable and control variables 

In [ ]:

file_path = r"C:\Users\Dom3n\Downloads\Control Variables.xlsx"

# Dataframe with the variables of interest 
mbase = pd.read_excel(file_path)
mbase.head()


In [ ]:
#Descriptive statistics of the variables 
mbase.describe()

In [ ]:
# Color of the plots 
color = '#1f77b4'  

# Plot S&P 500 
plt.figure()
plt.plot(mbase['date'], mbase['interpolated_returns'], color=color)
plt.xlabel('Date')
plt.ylabel('S&P 500')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()  
plt.show()

In [ ]:


# Plot VIX
plt.figure()
plt.plot(mbase['date'], mbase['VIX_inter'], color=color)
plt.xlabel('Date')
plt.ylabel('VIX')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Date 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()  
plt.show()

# Plot Bond 10 Years
plt.figure()
plt.plot(mbase['date'], mbase['bond_inter'], color=color)
plt.xlabel('Date')
plt.ylabel('Bond 10 Years')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1)) 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate() 
plt.show()

# Plot OFR FSI
plt.figure()
plt.plot(mbase['date'], mbase['OFR FSI_inter'], color=color)
plt.xlabel('Date')
plt.ylabel('OFR FSI')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1)) 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate() 
plt.show()

# Plot EPU
plt.figure()
plt.plot(mbase['date'], mbase['EPU_inter'], color=color)
plt.xlabel('Date')
plt.ylabel('EPU')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()  
plt.show()

Database with sentiment score , S&P 500 returns and control variables 

In [ ]:
#Code to merge the two dataframes 
base1 = pd.merge(df, mbase,  on='date')
base1.head()

In [ ]:

base1['date'] = pd.to_datetime(df['date'])
# Calculate the average Sentiment Score per day
average_sentiment = base1.groupby('date')['Sentiment Score'].var().reset_index()
# Rename the column for clarity
average_sentiment.rename(columns={'Sentiment Score': 'Average Sentiment'}, inplace=True)
# merge the daily average Sentiment Score with the original
base2= base1.merge(average_sentiment, on='date', how='left')


In [ ]:
# Save the data in a CSV file in the specified path
output_path = r'C:\Users\Dom3n\Downloads\base3.xlsx'
base2.to_excel(output_path, index=False)

In [ ]:

# Remove duplicate rows to keep one row per date
base3= base2.drop_duplicates(subset=['date'])
base3 = base3.dropna(subset=['date'])
base3.head()


In [ ]:
base4 = base3[base3['Sentiment Category'] != 'Positive']
base4.head()
